In [ ]:
import json
import os.path
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import functools
import matplotlib.pyplot as plt
from config import Portfolio
from downloadEquities import downloadEquities
from parsePortfolio import parsePortfolio

In [ ]:
# Config

useDataOverYears = 5 # 1y,2y,5y,10y
endDate = datetime.date.today() # only influences parsing atm, not collection
portfolioFile = 'portfolio.json'

In [ ]:
# Load and Initialise Portfolio
def loadPortfolio() -> Portfolio:
    sample = 'portfolioSample.json'
    useSample = False
    if not os.path.isfile(portfolioFile):
        print("Using sample data.\nPlease create a portfolio.json to use custom input.")
        useSample = True
    with open(sample if useSample else portfolioFile, 'r') as infile:
        return json.load(infile)

portfolio = loadPortfolio()
portfolioEquities = portfolio["equities"]
portfolioOther = portfolio["other"]

In [ ]:
# Download equities
    
downloadEquities(portfolioEquities, useDataOverYears)

In [ ]:
# Parse portfolio data

portfolioDf = parsePortfolio(portfolio=portfolio, useDataOverYears=useDataOverYears, endDate=endDate)
portfolioDf.sort_values(by=['name'], inplace=True)
portfolioLabels = portfolioDf['name'].sort_values(ascending=True)
portfolioWeights = portfolioDf['weight']
portfolioChanges = portfolioDf['changePerAnnum'] * 100
portfolioWeightedChanges = portfolioDf['weightedReturnPerAnnum'] * 100
portfolioShares = portfolioDf['sharePriceChangePerAnnum'] * 100
portfolioDividends = portfolioDf['estimatedDividendBonusToChange'] * 100
totalExpectedReturn = portfolioWeightedChanges.sum()

display(portfolioDf)


# Portfolio Unweighted Performance
fig = plt.figure(figsize=(10,5))
plt.bar(
    portfolioLabels,
    portfolioShares,
)
plt.bar(
    portfolioLabels,
    portfolioDividends,
    bottom=portfolioShares
)
plt.title("Unweighted Change Per Annum", pad=15)
plt.ylabel('Percentage')
plt.legend(['Share price', 'Dividends'])
plt.xticks(portfolioLabels)

plt.show()


In [ ]:
def makePortfolioPie(values, title):
    cmap = plt.get_cmap("Set3")
    colorSet = cmap(np.arange(len(portfolioDf)))
    fig = plt.figure(figsize=(6,6))

    plt.pie(
        values, 
        labels=portfolioLabels, 
        colors=colorSet,
        autopct='%1.1f%%', 
        shadow=True, 
        startangle=130
    )
    plt.axis('equal')
    plt.title(title, pad=15)
    plt.tight_layout()
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    plt.show()

makePortfolioPie(
    values=portfolioWeights, 
    title="Portfolio Composition"
)

makePortfolioPie(
    values=portfolioWeightedChanges / totalExpectedReturn, 
    title="Investment Return by Portfolio Composition"
)
